In [1]:

# -*- coding: utf-8 -*-
"""Symbolic Seismic Tracker (Modular & Extended)

Symbolic entropy, φ-word detection, attractor residue, and prime-interval resonance.
Modular parameters for region and time window.
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from datetime import datetime
from obspy import UTCDateTime
from sympy import isprime

# Configurable parameters for region and time
REGION = {
    "name": "Turkey-Syria Earthquake 2023",
    "start": "2022-12-23",
    "end": "2023-02-11",
    "lat_min": 35.5,
    "lat_max": 38.5,
    "lon_min": 35.5,
    "lon_max": 38.5,
    "minmag": 2.0
}

def fetch_earthquake_data(region):
    try:
        url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
        params = {
            "format": "geojson",
            "starttime": region["start"],
            "endtime": region["end"],
            "minlatitude": region["lat_min"],
            "maxlatitude": region["lat_max"],
            "minlongitude": region["lon_min"],
            "maxlongitude": region["lon_max"],
            "minmagnitude": region["minmag"]
        }
        response = requests.get(url, params=params)
        data = response.json()
        records = []
        for feat in data["features"]:
            props = feat["properties"]
            time = datetime.utcfromtimestamp(props["time"] / 1000.0)
            records.append([time, props["mag"]])
        return pd.DataFrame(records, columns=["timestamp", "magnitude"])
    except Exception as e:
        print("Error fetching data:", e)
        return pd.DataFrame()

# Load and preprocess data
df = fetch_earthquake_data(REGION)
if df.empty:
    raise SystemExit("No data available for region and time window.")

df = df.sort_values("timestamp").reset_index(drop=True)
df["delta_mag"] = df["magnitude"].diff()
df["binary_delta"] = (df["delta_mag"] > 0).astype(int)

# φ-word detection and alignment
phi_motif = "01001010"
bitstream = ''.join(df["binary_delta"].astype(str))
motif_indices = [i for i in range(len(bitstream) - len(phi_motif)) if bitstream[i:i+8] == phi_motif]
motif_timestamps = [df["timestamp"].iloc[i + len(phi_motif)//2] for i in motif_indices]
print(f"Occurrences of φ-word motif (01001010): {len(motif_indices)}")

# Entropy functions
def shannon_entropy(seq):
    p = np.mean(seq)
    if p in [0, 1]: return 0
    return -(p * np.log2(p) + (1 - p) * np.log2(1 - p))

df["entropy_5"] = [np.nan]*4 + [shannon_entropy(df["binary_delta"].iloc[i:i+5]) for i in range(len(df)-4)]
df["entropy_7"] = [np.nan]*6 + [shannon_entropy(df["binary_delta"].iloc[i:i+7]) for i in range(len(df)-6)]
df["entropy_vel_5"] = df["entropy_5"].diff()
df["entropy_acc_5"] = df["entropy_vel_5"].diff()

# Attractor residue
ATTRACTORS = {
    "alpha_inv": 1/137.035999,
    "phi_sq": (1.6180339887)**2,
    "pi_over_2": np.pi / 2
}
df["residue_alpha"] = abs(df["entropy_5"] - ATTRACTORS["alpha_inv"])
df["residue_phi2"] = abs(df["entropy_5"] - ATTRACTORS["phi_sq"])
df["residue_pi2"] = abs(df["entropy_5"] - ATTRACTORS["pi_over_2"])

# Prime-interval detection
df["timestamp_shift"] = df["timestamp"].shift(1)
df["interval_sec"] = (df["timestamp"] - df["timestamp_shift"]).dt.total_seconds()
df["is_prime_interval"] = df["interval_sec"].fillna(0).astype(int).apply(isprime)
prime_count = df["is_prime_interval"].sum()
print(f"Quake intervals with prime-second spacing: {prime_count}")

# Export to CSV
output_file = f"symbolic_metrics_{REGION['name'].replace(' ', '_')}.csv"
df.to_csv(output_file, index=False)
print(f"Symbolic metrics saved to: {output_file}")

# φ-word motif timestamps (optional log)
motif_log = pd.DataFrame({"timestamp": motif_timestamps})
motif_log.to_csv("phi_motif_timestamps.csv", index=False)


Occurrences of φ-word motif (01001010): 7
Quake intervals with prime-second spacing: 47
Symbolic metrics saved to: symbolic_metrics_Turkey-Syria_Earthquake_2023.csv


In [2]:
%pip install obspy